In [32]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation pipelines
# General augmentation
common_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomTranslation(0.1, 0.1),
    keras.layers.RandomContrast(0.2),
])

# Stronger augmentation for minority class
strong_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.3),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomTranslation(0.2, 0.2),
    keras.layers.RandomContrast(0.3),
    keras.layers.RandomBrightness(0.3),
    keras.layers.RandomWidth(0.2),
    keras.layers.RandomHeight(0.2),
])

# Normalization layer
normalization_layer = keras.layers.Rescaling(1./255)

# Load data
def preprocess_dataset(dataset, class_id):
    if class_id == 2:  # Apply stronger augmentation to minority class
        return dataset.map(lambda x, y: (strong_augmentation(normalization_layer(x)), y))
    else:  # Apply common augmentation to other classes
        return dataset.map(lambda x, y: (common_augmentation(normalization_layer(x)), y))

train_ds = keras.utils.image_dataset_from_directory(
    "dataset-v2/dataset_classified_split/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    color_mode="rgb"
)
val_ds = keras.utils.image_dataset_from_directory(
    "dataset-v2/dataset_classified_split/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    color_mode="rgb"
)
test_ds = keras.utils.image_dataset_from_directory(
    "dataset-v2/dataset_classified_split/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    color_mode="rgb"
)

# Apply class-specific augmentation
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 780 files belonging to 3 classes.
Found 196 files belonging to 3 classes.
Found 196 files belonging to 3 classes.
Found 245 files belonging to 3 classes.
Found 245 files belonging to 3 classes.


In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import ResNet50

# Build Hybrid ResNet + ViT model for burn classification
def build_resnet_vit(input_shape=(224, 224, 3), num_classes=3, embed_dim=256, num_heads=4, transformer_layers=2, mlp_dim=512, dropout=0.3):
    inputs = Input(shape=input_shape)
    # CNN Backbone (ResNet50, no top)
    resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    resnet.trainable = False
    x = resnet(inputs)
    # Flatten spatial grid to sequence of patches
    patches = layers.Reshape((49, 2048))(x)
    # Linear projection to transformer dimension
    x = layers.Dense(embed_dim)(patches)
    # Add positional encoding
    positions = tf.range(start=0, limit=49, delta=1)
    pos_embed = layers.Embedding(input_dim=49, output_dim=embed_dim)(positions)
    x = x + tf.reshape(pos_embed, [1, 49, embed_dim])
    # Transformer encoder blocks
    for _ in range(transformer_layers):
        # Layer norm
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Multi-head attention
        attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)(x1, x1)
        x2 = layers.Add()([attn, x])
        # Layer norm
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP
        mlp = layers.Dense(mlp_dim, activation='gelu')(x3)
        mlp = layers.Dropout(dropout)(mlp)
        mlp = layers.Dense(embed_dim)(mlp)
        mlp = layers.Dropout(dropout)(mlp)
        x = layers.Add()([x2, mlp])
    # Global average pooling over sequence
    x = layers.GlobalAveragePooling1D()(x)
    # Classification head
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)

# Build the model
model = build_resnet_vit()
model.summary()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ input_layer_23[0… │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_5 (Reshape) │ (None, 49, 2048)  │          0 │ resnet50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 49, 256)   │    524,544 │ reshape_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_25 (Add)        │ (None, 49, 256)   │          0 │ dense_35[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 49, 256)   │        512 │ add_25[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 49, 256)   │    263,168 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_26 (Add)        │ (None, 49, 256)   │          0 │ multi_head_atten… │
│                     │                   │            │ add_25[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 49, 256)   │        512 │ add_26[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 49, 512)   │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_41          │ (None, 49, 512)   │          0 │ dense_36[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 49, 256)   │    131,328 │ dropout_41[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_42          │ (None, 49, 256)   │          0 │ dense_37[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_27 (Add)        │ (None, 49, 256)   │          0 │ add_26[0][0],     │
│                     │                   │            │ dropout_42[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 49, 256)   │        512 │ add_27[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 49, 256)   │    263,168 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_28 (Add)        │ (None, 49, 256)   │          0 │ multi_head_atten… │
│                     │                   │            │ add_27[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 49, 256)   │        512 │ add_28[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 25,199,747 (96.13 MB)

 Trainable params: 1,612,035 (6.15 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [34]:
import numpy as np

# MixUp augmentation - fixed version
@tf.function
def mixup(batch_x, batch_y, alpha=0.2):
    batch_size = tf.shape(batch_x)[0]
    lambda_val = tf.random.uniform([], 0, 1)
    index = tf.random.shuffle(tf.range(batch_size))

    mixed_x = lambda_val * batch_x + (1 - lambda_val) * tf.gather(batch_x, index)
    mixed_y = lambda_val * tf.cast(batch_y, tf.float32) + (1 - lambda_val) * tf.cast(tf.gather(batch_y, index), tf.float32)
    return mixed_x, mixed_y

# CutMix augmentation - fixed version
@tf.function
def cutmix(batch_x, batch_y, alpha=1.0):
    batch_size = tf.shape(batch_x)[0]
    lambda_val = tf.random.uniform([], 0, 1)
    index = tf.random.shuffle(tf.range(batch_size))

    height, width = tf.shape(batch_x)[1], tf.shape(batch_x)[2]

    # Calculate cut size
    cut_ratio = tf.sqrt(1 - lambda_val)
    cut_w = tf.cast(tf.cast(width, tf.float32) * cut_ratio, tf.int32)
    cut_h = tf.cast(tf.cast(height, tf.float32) * cut_ratio, tf.int32)

    # Random center point
    cx = tf.random.uniform([], cut_w // 2, width - cut_w // 2, dtype=tf.int32)
    cy = tf.random.uniform([], cut_h // 2, height - cut_h // 2, dtype=tf.int32)

    # Create mask
    y_indices = tf.range(height)
    x_indices = tf.range(width)
    yy, xx = tf.meshgrid(y_indices, x_indices, indexing='ij')

    mask = tf.logical_and(
        tf.logical_and(yy >= cy - cut_h // 2, yy < cy + cut_h // 2),
        tf.logical_and(xx >= cx - cut_w // 2, xx < cx + cut_w // 2)
    )
    mask = tf.cast(mask, tf.float32)
    mask = tf.expand_dims(mask, 0)  # Add batch dimension
    mask = tf.expand_dims(mask, -1)  # Add channel dimension
    mask = tf.tile(mask, [batch_size, 1, 1, 3])  # Broadcast to all batches and channels

    mixed_x = batch_x * (1 - mask) + tf.gather(batch_x, index) * mask
    mixed_y = lambda_val * tf.cast(batch_y, tf.float32) + (1 - lambda_val) * tf.cast(tf.gather(batch_y, index), tf.float32)

    return mixed_x, mixed_y

# Simplified augmentation function that works with batches
def apply_combined_augmentation(x, y):
    # Apply basic augmentation first
    x = common_augmentation(x)

    # Then randomly apply mixup or cutmix to the batch
    rand_val = tf.random.uniform([])
    if rand_val > 0.7:  # 30% chance for mixup
        return mixup(x, y)
    elif rand_val > 0.4:  # 30% chance for cutmix
        return cutmix(x, y)
    else:  # 40% chance for no additional augmentation
        return x, y

In [35]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1, 2]),  # Convert to numpy array
    y=np.array([0] * 339 + [1] * 312 + [2] * 129)  # Convert to numpy array
)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]
EPOCHS = 30

# Start with simpler augmentation - just basic augmentation without mixup/cutmix
# Apply basic augmentation to training data
train_ds_augmented = train_ds.map(lambda x, y: (common_augmentation(x), y))

# Train the model
history = model.fit(
    train_ds_augmented,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2923 - loss: 1.8677 - val_accuracy: 0.3980 - val_loss: 1.0433 - learning_rate: 0.0010
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2923 - loss: 1.8677 - val_accuracy: 0.3980 - val_loss: 1.0433 - learning_rate: 0.0010
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3333 - loss: 1.2075 - val_accuracy: 0.4388 - val_loss: 1.0879 - learning_rate: 0.0010
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3333 - loss: 1.2075 - val_accuracy: 0.4388 - val_loss: 1.0879 - learning_rate: 0.0010
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3333 - loss: 1.1525 - val_accuracy: 0.3980 - val_loss: 1.1007 - learning_rate: 0.0010
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3333 - loss: 1.1525 - val_accuracy: 0.3980 - val_loss: 1.1007 - learning_rate: 0.0010
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3423 - loss: 1.1363 - val_accuracy: 

In [37]:
# Stage 2: Fine-tuning the ResNet backbone
# Unfreeze the last 30 layers of the ResNet backbone
for layer in model.layers[1].layers[-30:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with the updated configuration
history_stage2 = model.fit(
    train_ds_augmented,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)

# Evaluate the model after fine-tuning
test_loss_stage2, test_acc_stage2 = model.evaluate(test_ds)
print(f"Stage 2 Test accuracy: {test_acc_stage2:.4f}")

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3679 - loss: 1.1750 - val_accuracy: 0.3980 - val_loss: 1.0435 - learning_rate: 1.0000e-05
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3679 - loss: 1.1750 - val_accuracy: 0.3980 - val_loss: 1.0435 - learning_rate: 1.0000e-05
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4179 - loss: 1.0802 - val_accuracy: 0.3980 - val_loss: 1.0490 - learning_rate: 1.0000e-05
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4179 - loss: 1.0802 - val_accuracy: 0.3980 - val_loss: 1.0490 - learning_rate: 1.0000e-05
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.4295 - loss: 1.0661 - val_accuracy: 0.3776 - val_loss: 1.0501 - learning_rate: 1.0000e-05
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.4295 - loss: 1.0661 - val_accuracy: 0.3776 - val_loss: 1.0501 - learning_rate: 1.0000e-05
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.3923 - loss: